<a href="https://colab.research.google.com/github/sri123vk/Movie-Recommendation-System-Movielens-dataset/blob/main/Autoencoders_MRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
  
  
uploaded = files.upload()

In [ ]:

#Import Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


#Dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

#training set and the test set
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

#number of users and movies
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

# Converting the data into an array 
#users-rows
#movies-columns
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

#Training SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))
  

#Testing SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s)) 




epoch: 1loss: tensor(1.7721)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0310)
epoch: 6loss: tensor(1.0264)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0217)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0195)
epoch: 11loss: tensor(1.0188)
epoch: 12loss: tensor(1.0183)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0172)
epoch: 16loss: tensor(1.0171)
epoch: 17loss: tensor(1.0166)
epoch: 18loss: tensor(1.0166)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0161)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0156)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0151)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0137)
epoch: 30loss: tensor(1.0118)
epoch: 31loss: tensor(1.0099)
epoch: 32loss: tensor(1.0099)
epoch: 33loss: tensor(1.0054)
epoch: 34loss: tens

In [ ]:
#Prediction
user_id = 0
movie_title = movies.iloc[:nb_movies, 1:2]
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
result_array = np.hstack([movie_title, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=["Movie","Target Rating","Predicted"])
predicted

array([[3.7755675],
       [3.2990217],
       [2.7974095],
       ...,
       [1.9930065],
       [3.2633061],
       [3.0441594]], dtype=float32)

In [ ]:
result_df.head()

,Movie,Target Rating,Predicted
0,GoldenEye (1995),3,3.82773
1,Dracula: Dead and Loving It (1995),5,4.39078
2,Nixon (1995),5,3.81526
3,Sense and Sensibility (1995),3,3.22925
4,Money Train (1995),4,3.30812
